In [1]:
import os
import sys
import controlflow as cf

from agent_utils import *
from agent_tools import *

from extensive_search import run_research
from rich import print as pprint
FLASH2_MODEL = "google/" + config.FLASH2_MODEL


In [3]:
response = perplexity_deep_research("How much power does the B200 system from Nvidia need and how much is the cooling water consumption in liters/Wh?")

ChatCompletion(id='cac72207-c2a2-4031-80a1-51fb597bd22d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="<think>\nOkay, let's tackle this query about the NVIDIA B200 system's power requirements and cooling water consumption. First, I need to gather all the relevant information from the provided search results.\n\nLooking at the search results, several sources mention the power consumption of the B200. Result [1] states that the B200's power consumption is over 40% higher than the H100, which is 700W, so that would put the B200 around 980W or more. Result [7] mentions the GB200 (which includes B200 GPUs) has a per GPU power consumption of 1000W, and result [13] directly states the B200 GPU reaches 1000W. There's a bit of variation here, but it seems the B200 GPU itself is around 1000-1200W depending on cooling method (air vs liquid), as per [8].\n\nNow, for the entire system. The DGX B200 system's power consumption is mentioned in [3

In [9]:
content = response.choices[0].message.content

cprint(content)

Looking at the search results, several sources mention the power consumption of the B200. Result [1] states that   
the B200's power consumption is over 40% higher than the H100, which is 700W, so that would put the B200 around    
980W or more. Result [7] mentions the GB200 (which includes B200 GPUs) has a per GPU power consumption of 1000W,   
and result [13] directly states the B200 GPU reaches 1000W. There's a bit of variation here, but it seems the B200 
GPU itself is around 1000-1200W depending on cooling method (air vs liquid), as per [8].                           

Now, for the entire system. The DGX B200 system's power consumption is mentioned in [3], [4], and [5]. Result [5]  
says the DGX B200 requires approximately 14.3 kW per system. However, [3] breaks down the DGX H100 at 10.2kW and   
the B200 as similar but with a 5-10% increase, which would be around 11-15kW. But [5] is more specific with 14.3kW,
so I'll take that as the system-level power. Then, when considering a rack with multiple systems, [2] mentions 4   
systems per rack at 57.2 kW, but [5] states 60 kW per rack. The GB200 NVL72 rack in [6] and [10] is even higher at 
120kW, but that's a different configuration.                                                                       

Cooling water consumption is trickier. Result [12] mentions coolant enters the rack at 2 liters per second, exiting
20°C warmer. To find liters per Wh, I need to calculate the thermal energy removed by the water. The specific heat 
capacity of water is 4.186 J/g°C. If the flow rate is 2 L/s (which is 2000 g/s), and the temperature rise is 20°C, 
then the heat removed per second is 2000g * 4.186 J/g°C * 20°C = 167,440 J/s or 167.44 kW. If the rack's total     
power is 120kW (from [6]), then the water is removing 167.44 kW of heat. However, this seems more than the rack's  
power, which might be due to inefficiencies or additional cooling. To find liters per Wh, convert 2 L/s to L/h     
(7200 L/h) and divide by the power in kW (120kW), giving 60 L/kWh. But this needs cross-verification with other    
sources. Result [9] mentions reducing water use, but doesn't give exact numbers. [11] also discusses liquid cooling
requirements but no specific liters/Wh.                                                                            


Putting it all together, the B200 GPU is around 1000-1200W, the DGX B200 system is ~14.3kW, and rack configurations
vary from 60kW to 140kW. Cooling water consumption for a high-end rack like GB200 NVL72 is 2 L/s, which converts to
approximately 60 liters per kWh. I need to structure this into sections, making sure to cite each source           
appropriately.                                                                                                     

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                               Power and Cooling Analysis of NVIDIA's B200 System                                ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The rapid advancement of AI computing hardware has brought NVIDIA's Blackwell architecture into sharp focus,       
particularly regarding its power demands and thermal management requirements. This report provides a comprehensive 
analysis of the NVIDIA B200 system's power consumption characteristics and the associated cooling water            
requirements, synthesizing data from technical specifications, industry reports, and engineering analyses.         


                                     Power Consumption Profile of B200 Systems                                     

                                          GPU-Level Power Characteristics                                          

At the core of NVIDIA's Blackwell architecture lies the B200 GPU, which represents a significant leap in thermal   
design power (TDP) compared to previous genera

In [19]:
rprint(response.citations)

[
    'https://longportapp.com/en/news/109092719',
    'https://www.fibermall.com/blog/nvidia-hgx-b200-cooling-solution.htm',
    'https://docs.nvidia.com/https:/docs.nvidia.com/nvidia-dgx-superpod-data-center-best-practices-with-dgx-b200.pd
f',
    'https://training.continuumlabs.ai/infrastructure/servers-and-chips/hopper-versus-blackwell',
    'https://www.youtube.com/watch?v=5vBnr084sYs',
    'https://futurumgroup.com/insights/cerebras-cs-3-bring-on-the-nvidia-blackwell-competition/',
    'https://viperatech.com/shop/nvidia-dgx-b200/',
    'https://www.anandtech.com/show/21310/nvidia-blackwell-architecture-and-b200b100-accelerators-announced-going-b
igger-with-smaller-data',
    'https://www.theregister.com/2024/03/18/nvidia_turns_up_the_ai/',
    'https://www.fibermall.com/blog/liquid-cooling-for-blackwell.htm',
    'https://w.media/what-the-arrival-of-nvidias-blackwell-means-for-data-centre-operations/',
    'https://www.fibermall.com/blog/nvidia-gb200-superchip.htm',
    'https://ir.supermicro.com/news/news-details/2025/Supermicro-Ramps-Full-Production-of-NVIDIA-Blackwell-Rack-Sca
le-Solutions-with-NVIDIA-HGX-B200/default.aspx',
    'https://www.techpowerup.com/forums/threads/nvidia-blackwells-high-power-consumption-drives-cooling-demands-liq
uid-cooling-penetration-expected-to-reach-10-by-late-2024.325046/',
    'https://docs.nvidia.com/dgx/dgxb200-user-guide/introduction-to-dgxb200.html',
    'https://blog.equinix.com/blog/2024/09/19/how-data-centers-use-water-and-how-were-working-to-use-water-responsi
bly/'
]

In [14]:
cprint(response.choices[0].message.content)

Starting with the power consumption. From the search results, there are several mentions of the B200's power draw. 
Result [2] says NVIDIA set TDP values at 1200W for liquid cooling and 1000W for air cooling. The DGX B200 system's 
max power usage is around 14.3kW according to [4] and [7]. Result [8] mentions that each GB200 module can run up to
2700W, which includes two GPUs and a CPU. So, breaking this down, each GPU in the B200 under liquid cooling hits   
1200W, and when combined in systems like the DGX B200 with eight GPUs, that adds up to significant power draw.     

Now, cooling water consumption. The user wants liters per Wh, which is a measure of water used per energy unit.    
Result [10] mentions transitioning from air to liquid cooling can reduce facility power by 27% and total site      
energy by 15.5%. However, specific water consumption rates are trickier. Result [16] discusses that liquid cooling 
doesn't necessarily increase water consumption compared to evaporative cooling, but exact figures aren't provided  
there. Result [5] talks about JetCool's technology reducing water use but doesn't give numbers.                    

Looking at result [10] again, Lenovo's Neptune system reduces power consumption by 40% compared to traditional     
methods. But how does that translate to water usage? Maybe looking at the cooling solutions described in [2], where
they discuss the OAI system's cold plate configurations and manifold designs. The CHx1000 CDU in [10] has a flow   
rate of 1.5 LPM/kW. If I convert that to liters per hour per kilowatt, it's 90 liters per hour per kW (1.5 L/min * 
60 min). So for a 1 kW load, that's 90 liters per hour. But wait, that seems high. Let me double-check: 1.5 liters 
per minute per kilowatt would be 90 liters per hour per kilowatt (1.5 * 60). So for a 1200W GPU, that's 1.2 kW * 90
L/h/kW = 108 L/h. But this is flow rate, not consumption. Since it's a closed-loop system, the actual water        
consumed would be minimal, mainly make-up water for any losses. Open-loop systems might use more, but many are     
closed.                                                                                                            

Result [12] mentions the GB200 NVL72 cabinet has a total power consumption of ~120kW and uses liquid cooling. If   
using the CHx1000's flow rate, 1.5 LPM/kW * 120 kW = 180 LPM, which is 10,800 liters per hour. But again, this is  
flow, not consumption. Actual water usage would depend on the cooling system type (open vs closed). Evaporative    
cooling towers consume water through evaporation and blowdown, while liquid cooling with CDUs in closed loops use  
less.                                                                                                              

From result [16], Equinix mentions that evaporative cooling consumes more water, but liquid cooling doesn't        
necessarily increase it. So maybe in closed-loop liquid systems, water consumption is minimal, just occasional     
replenishment. However, if they're using evaporative cooling for the CDU's heat rejection, that would add to water 
use.                                                                                                               

Putting this together: The B200 GPU itself has a TDP of up to 1200W under liquid cooling. Systems like DGX B200    
with eight GPUs draw ~14.3kW max. Cooling water consumption depends on the cooling method. Closed-loop liquid      
cooling might have minimal direct water use (just leakage/make-up), but if using evaporative towers for heat       
rejection, it could be higher. However, specific liters/Wh data isn't directly provided in the sources; I need to  
infer based on flow rates and system types.                                                                        

Wait result [10] also says Lenovo's Neptune system reduces power consumption by 40% compared to traditional        
cooling. But how does that relate to water? Maybe 

In [2]:
query = f"""
We need an detailed and extensive overview of existing systems for running pretrained model (e.g., transformer models) up to 100 million parameters (inference only).
To accomplish this, we plan to do a deep research by processing many webpages, PDFs, acticles etc. using LLM APIs (e.g., Google Gemini, GPT4o etc.).

We use an existing Python agent framework (e.g., PydanticAI, LangChain, Controlflow) to build the system.

We have access to different types of search tools:
- Google Search API (general search with time range search)
- Google Schoolar API (search papers)
- Google News API (search news articles)
- Could be expanded in the future.

For this task, we plan to build our own deep research agent system that uses LLMs to process a workflow.

The workflow has these basic working steps or tasks:
- Based on a search query (user input) generate a research plan/instruction and suitable search queries for the search tools.
- Use search APIs for generated search queries to collect information.
- Use a reasoning model (LLM + thinking) to select suitable URLs to extract information from.
- Use web crawlers to comvert HTML or PDF content to markdown text for the LLMs.
- For the first markdown document use an LLM to rate the relevance and quality of the document.
- For the first document also extract a summary of all relevant and important information.
- Save the document with rating and contectualized summary.
- For the second markdown document use the contectualized summary from the previous document, to genrate a rating and a new contextualized summary.
- Repeat the earlier steps for each downloaded document.
- Stop when a cerain document budget is reached.
- Use the original user query and the compressed/accumulated summary to create a report based on the generated research plan/instruction.

What do you think about this approach? How would you improve it?

"""

In [3]:
reasoningModel = ReasoningModel()
basicSearchModel = BasicSearchModel()


user_query = "We need a detailed and extensive overview of existing systems (e.g., ARM, FPGA, NPU, GPUs, etc.) for running pretrained model (e.g., transformer models) up to 100 million parameters (inference only)."

query = f"""
{user_query}

We can use different types of search tools:
- Google Search API (general search with time range search)
- Google Schoolar API (search papers)
- Google News API (search news articles)

Come up with suitable search queries for these tools.

The format should following this structure:

Google Search:
- query 1
- query 2
- etc.

Google Schoolar API:
- etc.


"""

if not (response := load_data("embedded_devices_search")):
    response = reasoningModel(query)
    save_data(response, "embedded_devices_search")
    cprint(response)

In [4]:

class SearchQueries(BaseModel):
    google_search: Optional[list[str]] = Field(description="The search queries from Google Search.")
    scholar_search: Optional[list[str]] = Field(description="The search queries from Google Scholar.")
    news_search: Optional[list[str]] = Field(description="The search queries from Google News.")

query_extraction = BasicAgent(result_type=SearchQueries, system_prompt="Extract the search queries from a given text (only the queries, nothing else).")


result = await query_extraction(f"{response}")

google_search_queries = result.data.google_search
scholar_search_queries = result.data.scholar_search
news_search_queries = result.data.news_search

/home/dplaia/Projekte/DeepResearch/agent_tools.py:39: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  return await self.agent.run(user_input)


## Google Search

In [6]:
url_list = []

In [9]:

if not (results := load_data("google_search_results")):
    results = []

    for google_search_query in google_search_queries:
        search_results = await google_general_search_async(google_search_query.replace("\"", ""), time_span = TimeSpan.YEAR)

        results.append(search_results)

    for result in results:
        organic = result['organic']

        urls = [t['link'] for t in organic]

        for url in urls:
            if not url in url_list:
                url_list.append(url)

    save_data(results, "google_search_results")

## Scholar Search

In [15]:
if not (results := load_data("scholar_search_results")):
    results = []

    for scholar_query in scholar_search_queries:
        search_results = await google_general_search_async(scholar_query.replace("\"", ""))

        results.append(search_results)

    for result in results:
        organic = result['organic']

        urls = [t['link'] for t in organic]

        for url in urls:
            if not url in url_list:
                url_list.append(url)
    save_data(results, "scholar_search_results")

## Scrape content

In [ ]:
if not (markdown_pages := load_data("markdown_pages")):

    markdown_pages = {}

    for url in url_list:
        print(url)

        try:
            markdown_text = await crawl4ai_website_async(url)
            markdown_pages[url] = markdown_text
        except Exception as e:
            print(e)
            continue

    save_data(markdown_pages, "markdown_pages")

In [29]:
memory = 0
for page in markdown_pages.values():
    memory += sys.getsizeof(page)

print(f"Memory needed: {memory/1024/1024:.2f} MB")

Memory needed: 24.91 MB


## Process content